In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [10]:
# Get postal code from Wikipedia
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695"
request = requests.get(url).text

# Passing the html to bs4
soup = BeautifulSoup(request, 'html.parser')

#print(soup.prettify())

In [11]:

#df1= pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M')[0]
#df1

In [12]:
table = soup.find('table').find_all('tr')

info = []

for row in table:
    info.append(row.text)
info

# Converting into a dataframe and assign column names
df = pd.DataFrame(info)
df = df[0].str.split('\n', expand=True)
df = df.rename(columns=df.iloc[0]).drop(df.index[0])

df.head(10)

,,Postcode,Borough,Neighbourhood,
1,,M1A,Not assigned,Not assigned,
2,,M2A,Not assigned,Not assigned,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,
6,,M5A,Downtown Toronto,Regent Park,
7,,M6A,North York,Lawrence Heights,
8,,M6A,North York,Lawrence Manor,
9,,M7A,Queen's Park,Not assigned,
10,,M8A,Not assigned,Not assigned,


### Cleansing the Data
Only cells that have an assigned borough are processed. More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [13]:
# Remove entries that do not have an assigned borough
df = df[df['Borough'] != 'Not assigned']
df.head()

,,Postcode,Borough,Neighbourhood,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,
6,,M5A,Downtown Toronto,Regent Park,
7,,M6A,North York,Lawrence Heights,


In [14]:
# Show entries that do not have an assigned neighbourhood
df[df['Neighbourhood'] == 'Not assigned']

,,Postcode,Borough,Neighbourhood,
9,,M7A,Queen's Park,Not assigned,


In [15]:
# Assigned the neighbourhood as the borough for the above
df.loc[ df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head(10)

,,Postcode,Borough,Neighbourhood,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,
6,,M5A,Downtown Toronto,Regent Park,
7,,M6A,North York,Lawrence Heights,
8,,M6A,North York,Lawrence Manor,
9,,M7A,Queen's Park,Queen's Park,
11,,M9A,Etobicoke,Islington Avenue,
12,,M1B,Scarborough,Rouge,
13,,M1B,Scarborough,Malvern,


In [16]:
# Joining Neighbourhood when shared the same postcode
df=df.groupby(['Postcode','Borough']).agg(', '.join)
df.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


### To show the number of rows in the dataframe

In [17]:
df.shape

(103, 1)

In [18]:
df.to_csv('toronto_postalcode.csv')